In [1]:
import pandas as pd 

In [2]:
train_MELD_df = pd.read_csv('../MELD_DATA/train_sent_emo.csv')
test_MELD_df = pd.read_csv('../MELD_DATA/test_sent_emo.csv')
val_MELD_df = pd.read_csv('../MELD_DATA/dev_sent_emo.csv')

In [3]:
train_MELD_df.head(20)

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"
5,6,"Now youll be heading a whole division, so you...",The Interviewer,neutral,neutral,0,5,8,21,"00:16:41,126","00:16:44,337"
6,7,I see.,Chandler,neutral,neutral,0,6,8,21,"00:16:48,800","00:16:51,886"
7,8,But therell be perhaps 30 people under you so...,The Interviewer,neutral,neutral,0,7,8,21,"00:16:48,800","00:16:54,514"
8,9,Good to know.,Chandler,neutral,neutral,0,8,8,21,"00:16:59,477","00:17:00,478"
9,10,We can go into detail,The Interviewer,neutral,neutral,0,9,8,21,"00:17:00,478","00:17:02,719"


In [4]:
print(f'Number of Utterances in the Training Set: ', len(train_MELD_df))
print(f'Number of Utterances in the Testing Set: ', len(test_MELD_df))
print(f'Number of Utterances in the Validation Set: ', len(val_MELD_df))

Number of Utterances in the Training Set:  9989
Number of Utterances in the Testing Set:  2610
Number of Utterances in the Validation Set:  1109


In [1]:
from embedding_functions import *

/blue/bonniejdorr/manuel.nunez/.conda/envs/VANESSA_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def create_dialogue_graphs(df):

    dialogues = df.groupby('Dialogue_ID')
    dialogue_graphs = []

    # Create a placeholder for null embeddings (assuming embeddings are vectors)
    null_embedding = [0] * 768  

    # To iterate over each group
    for _, dialogue in dialogues:

        participants = dialogue['Speaker'].unique()
        G = []

        for index, row in dialogue.iterrows():
        
            G_i = {'X': {}, 'Y': 'Uninitialized'}
            G_i['Y'] = row['Emotion']
            speaker = row['Speaker']

            # Initialize speaker nodes for each timestamp
            for p in participants:
                if p == speaker:
                    speaker_embedding = embedding_func(row['Utterance'])
                    speaker_edges = [f'{node}_{index}' for node in participants if node != speaker]
                    G_i['X'][f'{p}_{index}'] = {'embedding': speaker_embedding, 'edges': speaker_edges}
                else:    
                    G_i['X'][f'{p}_{index}'] = {'embedding': null_embedding, 'edges': []}

            G.append(G_i)

        dialogue_graphs.append(G)


    return dialogue_graphs

In [14]:
MELD_val_dialogue_graphs = create_dialogue_graphs(val_MELD_df)
MELD_train_dialogue_graphs = create_dialogue_graphs(train_MELD_df)
MELD_test_dialogue_graphs = create_dialogue_graphs(test_MELD_df)

In [8]:
import matplotlib.pyplot as plt


def plot_speakers_vs_utterances(df, title):
    dialogues = df.groupby('Dialogue_ID')

    num_speakers = []
    num_utterances = []

    # To iterate over each group
    for _, dialogue in dialogues:
        num_speakers.append(len(dialogue['Speaker'].unique()))
        num_utterances.append(len(dialogue))

    
    plt.figure(figsize=(8, 6))
    plt.scatter(num_utterances, num_speakers, color='blue')
    plt.title(f'Number of Speakers vs Number of Utterances ({title})')
    plt.xlabel('Number of Utterances')
    plt.ylabel('Number of Speakers')
    plt.grid(True)
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plot_speakers_vs_utterances(train_MELD_df, 'Training Set')
plot_speakers_vs_utterances(test_MELD_df, 'Testing Set')
plot_speakers_vs_utterances(val_MELD_df, 'Validation Set')

In [15]:
import pickle

with open('data/ERC/MELD/val_dialogue_graphs_new.pkl', 'wb') as f:
    pickle.dump(MELD_val_dialogue_graphs, f)

with open('data/ERC/MELD/train_dialogue_graphs_new.pkl', 'wb') as f:
    pickle.dump(MELD_train_dialogue_graphs, f)

with open('data/ERC/MELD/test_dialogue_graphs_new.pkl', 'wb') as f:
    pickle.dump(MELD_test_dialogue_graphs, f)

In [7]:
augmented_data = pd.read_csv('../MELD_DATA/Augmented_MELD_3000.csv')
MELD_augmented_dialogue_graphs = create_dialogue_graphs(augmented_data)

In [8]:
import pickle
with open('data/ERC/MELD/augmented_dialogue_graphs_3000.pkl', 'wb') as f:
    pickle.dump(MELD_augmented_dialogue_graphs, f)